In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import data
import transporter
import torch.utils.data

%load_ext autoreload
%autoreload 2

In [ ]:
np.random.seed(42)

batch_size = 32
image_channels = 3
k = 4
num_features = 32

feature_encoder = transporter.FeatureEncoder(image_channels)
pose_regressor = transporter.PoseRegressor(image_channels, k)
refine_net = transporter.RefineNet(image_channels)

model = transporter.Transporter(
    feature_encoder, pose_regressor, refine_net
)

model.load_state_dict(
    torch.load('checkpoints/model.pth', map_location='cpu')
)
model.eval();

In [ ]:
from data import Dataset, Sampler
from torchvision import transforms

transform = transforms.ToTensor()
dataset = Dataset('data', transform=transform)
sampler = Sampler(dataset)
loader = torch.utils.data.DataLoader(dataset,
                                     batch_size=16,
                                     sampler=sampler,
)

model.eval()
for xt, xtp1 in loader:
    break
    
target_keypoints = model.point_net(xtp1)
reconstruction = model(xt, xtp1)

In [ ]:
from matplotlib.patches import Circle
from utils import get_n_colors

fig, ax = plt.subplots(1, 5, figsize=(12, 4))
idx = 6
k_idx = 1
std = 0.1


feature_maps = transporter.spatial_softmax(target_keypoints)
gmap = transporter.gaussian_map(feature_maps, std)[idx, k_idx]

ax[0].imshow(xt[idx].permute([1, 2, 0]).detach().numpy(), cmap='gray')
ax[1].imshow(xtp1[idx].permute([1, 2, 0]).detach().numpy())
ax[2].imshow(reconstruction[idx].permute([1, 2, 0]).detach().numpy())
ax[3].imshow(gmap.detach().numpy(), cmap='gray', vmin=0, vmax=1)
ax[4].imshow(feature_maps[idx, k_idx].detach().numpy(), cmap='gray')

locs = transporter.compute_keypoint_location_mean(
    transporter.spatial_softmax(model.point_net(xtp1)))[idx]

colors = get_n_colors(len(locs))
for i, l in enumerate((locs + 1) / 2 * 80):
    ax[1].add_patch(Circle((l[1].item(), l[0].item()), 2, 
                           color=colors[i], alpha=0.5))

In [ ]:
from PIL import Image, ImageDraw

def get_keypoints(model, source_images):
    return transporter.compute_keypoint_location_mean(
        transporter.spatial_softmax(model.point_net(source_images)))
    return source_keypoints


def annotate_keypoints(img, kp, colors):
    draw = ImageDraw.Draw(img)

    for i, kp in enumerate(unnormalize_kp(kp_t, image_width)):
        y = kp.detach().numpy()[0]
        x = kp.detach().numpy()[1]
        r = 2
        draw.ellipse((x-r, y-r, x+r, y+r), tuple(colors[i]))
    
    return img

def unnormalize_kp(kp, img_width):
    return (kp + 1) / 2 * img_width

traj = torch.stack(dataset.get_trajectory(0))
keypoints = get_keypoints(model, traj)
image_width = traj.size(-1)


In [ ]:
fig = plt.figure(figsize=(12, 3))

colors = get_n_colors(k)
colors = [(int(color[0] * 255), int(color[1]*255), int(color[2]*255), 255) for color in colors]
for i, t in enumerate(range(0, 100, 20)):
    frame_t = traj[t].permute([1, 2, 0])
    kp_t = keypoints[t]
    im = Image.fromarray((frame_t.detach().numpy() * 255).astype('uint8'))
    annotate_keypoints(im, kp_t, colors)
    im = np.array(im)
    ax = fig.add_subplot(1, 5, i + 1)
    ax.imshow(im)
    ax.set_axis_off()
    ax.set_title('t = {}'.format(t))

fig.savefig('assets/timesteps.pdf', bbox_inches='tight')
fig.savefig('assets/timesteps.png', bbox_inches='tight')

In [ ]:
from PIL import Image, ImageDraw

traj = torch.stack(dataset.get_trajectory(0))
keypoints = get_keypoints(model, traj)
image_width = traj.size(-1)

def get_heatmaps(model, source_images, normalize=True):
    if normalize:
        return transporter.spatial_softmax(model.point_net(source_images))
    return model.point_net(source_images)


In [ ]:
fig, ax = plt.subplots(1, k+1, figsize=(12, 3))

idx = 20

ax[0].imshow(traj[idx].permute([1, 2, 0]).detach().numpy())
ax[0].set_axis_off()

hm = get_heatmaps(model, traj, normalize=False)[idx].detach().numpy()

for i, m in enumerate(hm):
    ax[i+1].imshow(m, cmap='gray')
    ax[i+1].set_axis_off()
    
fig.savefig('assets/heatmaps_unnormalized.pdf', bbox_inches='tight')
fig.savefig('assets/heatmaps_unnormalized.png', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1, k+1, figsize=(12, 3))

idx = 20

ax[0].imshow(traj[idx].permute([1, 2, 0]).detach().numpy())
ax[0].set_axis_off()


hm = get_heatmaps(model, traj, normalize=True)[idx].detach().numpy()

for i, m in enumerate(hm):
    ax[i+1].imshow(m, cmap='gray')
    ax[i+1].set_axis_off()
fig.savefig('assets/heatmaps_normalized.pdf', bbox_inches='tight')
fig.savefig('assets/heatmaps_normalized.png', bbox_inches='tight')